#Scrapping des textes sur genius

In [0]:
!pip install lyricsgenius

     |████████████████████████████████| 92kB 4.5MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


Import des differentes librairies

In [0]:
import lyricsgenius
from time import sleep
import pandas as pd
import json
import nltk
nltk.download('punkt')

In [0]:
genius = lyricsgenius.Genius("yOow6voYeOCzfj0NjurGmKk9hyVgfrlDWWvZLxO5HE1S0AEKX6pECRH0NUgDliYw")
genius.remove_section_headers = True
genius.excluded_terms = ["Livret","Remix","Edit","Pochette","Intro","Outro","Freestyle"]
artists = ["Booba","Damso","Guizmo","Kaaris","Lomepal","Nepal","Nekfeu","Orelsan","PNL","SCH","Vald"]
for art in artists:
  artist = genius.search_artist(art, sort="title")
  artist.save_lyrics()
  sleep(900)

Searching for songs by Booba...

Song 1: "03/02/15"
Song 2: "0.9"
Song 3: "100-8 Zoo"
Song 4: "113"
Song 5: "1.8.7"
Song 6: "2.0"
Song 7: "2Pac"
Song 8: "3G"
Song 9: "45 Scientific"
Song 10: "4G"
Song 11: "7 Vie Là"
Song 12: "92i Veyron"
Song 13: "92izi"
Song 14: "A3"
Song 15: "A4"
"A4 - Remix" is not valid. Skipping.
Song 16: "Abracadabra"
Song 17: "A.C. Milan"
Song 18: "À la folie"
Song 19: "All Set"
Song 20: "Alter égo"
Song 21: "Animals"
Song 22: "Arc-en-ciel"
Song 23: "Attila"
Song 24: "Au bout des rêves"
"Au bout des rêves (Freestyle)" is not valid. Skipping.
Song 25: "Au fond de la classe"
Song 26: "Avant de partir"
"Avant de partir (Edit)" is not valid. Skipping.
Song 27: "B2OBA"
Song 28: "Baby"
Song 29: "Bad Boy Street"
Song 30: "Bakel City Gang"
Song 31: "Bâtiment C"
Song 32: "Batiment C, part. II"
Song 33: "BB"
Song 34: "Bellucci"
Song 35: "Billets verts"
Song 36: "Billets violets"
"B.O (Freestyle Radio)" is not valid. Skipping.
Song 37: "Boîte vocale"
"Boîte vocale (Freesty

In [0]:
def prepro(datalist,fnames):
  datalist = list(datalist)
  fnames = list(fnames)
  for item,fname in zip(datalist,fnames):
    with open(item) as f:
      data = json.load(f)

    artist_dict = {}
    
    def collectSongData(adic):
      dps = list()
      title = adic['title'] #song title
      lyrics = adic['lyrics'] #song lyrics
      dps.append((title,lyrics)) #append all to one tuple list
      artist_dict[title] = dps #assign list to song dictionary entry named after song title

    for i in range(len(data['songs'])):
      collectSongData(data['songs'][i])
    lyrics = pd.DataFrame.from_dict(artist_dict, orient="index").reset_index()
    lyrics.columns = ["title","lyrics"]
    for i in range(len(lyrics.lyrics)):
      lyrics.lyrics[i]=lyrics.lyrics[i][1:]
    lyrics.lyrics = lyrics.lyrics.apply(lambda x: ''.join(x))
    lyrics_clean = lyrics.lyrics.str.split("\n\n", expand=True)

    mypunct = str('!"#$%&\'()*+,-./;<=>?@\\^’_`{|}~')

    for col in lyrics_clean.columns:
      lyrics_clean[col] = lyrics_clean[col].str.replace("[{}]".format(mypunct)," ")
      lyrics_clean[col] = lyrics_clean[col] + "\n"
      lyrics_clean[col] = lyrics_clean[col].str.lower()

    lyrics_clean.fillna('', inplace=True)
    lyrics["lyrics_clean"]=""

    for index, row in lyrics_clean.iterrows():
      lyrics["lyrics_clean"][index] = " ".join(row)

    lyrics.to_csv("{}".format(fname))

datalist = ["Lyrics_Booba.json","Lyrics_Damso.json","Lyrics_Guizmo.json","Lyrics_Kaaris.json","Lyrics_Lomepal.json",
            "Lyrics_Nekfeu.json","Lyrics_Népal.json","Lyrics_OrelSan.json","Lyrics_PNL.json","Lyrics_SCH.json","Lyrics_Vald.json"]
fnames = ["booba.csv","damso.csv","guizmo.csv","kaaris.csv","lomepal.csv","nekfeu.csv","nepal.csv","orelsan.csv","pnl.csv","sch.csv","vald.csv"]

prepro(datalist,fnames)

##Creation des différents datasets

In [0]:
booba = pd.read_csv("booba.csv")
damso = pd.read_csv("damso.csv")
guizmo = pd.read_csv("guizmo.csv")
kaaris = pd.read_csv("kaaris.csv")
lomepal = pd.read_csv("lomepal.csv")
nekfeu = pd.read_csv("nekfeu.csv")
nepal = pd.read_csv("nepal.csv")
orelsan = pd.read_csv("orelsan.csv")
pnl = pd.read_csv("pnl.csv")
sch = pd.read_csv("sch.csv")
vald = pd.read_csv("vald.csv")

#Traitement du texte

In [0]:
def split_lines(lyrics):
  lyrics.lyrics = lyrics.lyrics.apply(lambda x: str(x))
  liste = []
  for i in range(len(lyrics)):
    lyric = lyrics.lyrics[i].splitlines()
    liste+=lyric
  dataset = pd.DataFrame(liste, columns=['sentences'])
  return(dataset)

In [0]:
dataset = split_lines(nekfeu)
initial_len = len(dataset)
print('Nous avons {} lignes initialement'.format(initial_len))

Nous avons 8133 lignes initialement


In [0]:
def drop_null_sentence(dataset):
  drop = list()
  for i in range(len(dataset.sentences)):
    if len(dataset.sentences[i].split(' '))<2:
      drop.append(i)
  dataset = dataset.drop(index=drop)
  dataset = dataset.reset_index()
  return(dataset)

In [0]:
dataset = drop_null_sentence(dataset)
print('Il nous reste {} lignes ({}%)'.format(len(dataset),round(len(dataset)/initial_len*100,1)))
data = dataset

Il nous reste 7480 lignes (92.0%)


In [0]:
inputdict = {'?':'','...':'','!':'','(':'', ')':'','"':'','/':'','.':' ',',':'',':':''}

In [0]:
#Suppression de la ponctuation et des caractères de l'inputdict
def text_harmonization(dataset):
  sentences = dataset.sentences.apply(lambda x: x.lower())
  s=' '
  phrase_clean = []
  for sentence in sentences:
    for word, initial in inputdict.items():
      sentence = sentence.replace(word.lower(), initial)
    phrase_clean.append(sentence)
  dataset_harmonized = pd.DataFrame(phrase_clean, columns=["sentences"])
  return(dataset_harmonized)

In [0]:
dataset = text_harmonization(dataset)

In [0]:
# Python program to check if two to get unique values from list
# using traversal function to get unique values

def unique(liste):
    # intilize a null list
    unique_list = []
    # traverse for all elements
    for x in liste:
        # check if exists in unique_list or not
        if x not in unique_list:
            unique_list.append(x)
    return(unique_list)

In [0]:
def del_word_repeat(dataset):
  space = ' '
  data = []
  for sentence in dataset.sentences:
    words = sentence.split()
    if len(words)>0:  
      new_sentence = [words[0]]
    for i in range(1,len(words)):
      if words[i-1]!=words[i]:
        new_sentence.append(words[i])
    new_sentence = space.join(new_sentence)
    data.append((new_sentence))
    dataset = pd.DataFrame(data, columns=['sentences'])
  return(dataset)

In [0]:
dataset = del_word_repeat(dataset)

In [0]:
def del_repetition(dataset):
  sentences = dataset.sentences
  clean = []
  for phrase in sentences:
    split = phrase.split(' ')
    if len(unique(split)) > len(split)/2:
      clean.append(split)
  for i in range(len(clean)):
    s = ' '
    clean[i] = s.join(clean[i])
  return(pd.DataFrame(clean, columns=['sentences']))

In [0]:
dataset = del_repetition(dataset)

In [0]:
def del_sentences_repeat(dataset):
  sentences = dataset.sentences
  clean = []
  for i in range(len(sentences)-2):
    if sentences[i]!=sentences[i+1] and sentences[i]!=sentences[i+2]:
      clean.append(sentences[i])
  dataset = pd.DataFrame(clean, columns=['sentences'])
  return(dataset)

In [0]:
dataset = del_sentences_repeat(dataset)
print('Il nous reste {} lignes ({}%)'.format(len(dataset),round(len(dataset)/initial_len*100,1)))

Il nous reste 7077 lignes (87.0%)
